[View in Colaboratory](https://colab.research.google.com/github/dkatsios/semantic_segmentation/blob/master/voc2012.ipynb)

In [0]:
import numpy as np
import os
from shutil import unpack_archive
import cv2
from matplotlib import pyplot as plt
from IPython.display import Image
import PIL
from keras.optimizers import Adam
from time import time
from google.colab import files
from keras.models import load_model

### Download VOC 2012 dataset

In [0]:
# %mkdir semantic_segmentation
# %cd semantic_segmentation/
# !wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
# unpack_archive('VOCtrainval_11-May-2012.tar', './')

In [0]:
%cd /content/semantic_segmentation/VOCdevkit/VOC2012
!ls

In [0]:
imgs_folder = './JPEGImages/'
classes_folder = './SegmentationClass/'
train_list_path = './ImageSets/Segmentation/train.txt'
val_list_path = './ImageSets/Segmentation/val.txt'

### Import helpers and Model files

In [0]:
files.upload()

In [0]:
from voc2012_helpers import *
from atrusunet import  *

### Set parameters

In [0]:
img_shape = 512, 512, 3
filters = 64
segmentation_classes = 21
steps = 5
out_resized_levels = 4
kernel_sizes = [2, 3, 5]
batch_size = 4
epochs = 7
val_steps = 10

### Build generators

In [0]:
# train_lists, val_lists = get_lists_from_folders(train_list_path, val_list_path, imgs_folder, classes_folder)
# train_arrays = get_imgs_classes_arrays(*train_lists, img_shape)
# val_arrays = get_imgs_classes_arrays(*val_lists, img_shape)
# steps_per_epoch= len(train_lists[0]) // batch_size

In [0]:
train_gen = imgs_generator(*train_arrays, batch_size, out_resized_levels, segmentation_classes)
val_gen = imgs_generator(*val_arrays, batch_size, out_resized_levels, segmentation_classes)

### Build model

In [0]:
atrous_unet = AtrousUnet(img_shape, filters, segmentation_classes,
                         steps, out_resized_levels, kernel_sizes)

atrous_unet.build_model()
print('number of parameters:', atrous_unet.model.count_params())

### Compile model

In [0]:
optimizer = Adam(0.001)

losses = ['categorical_crossentropy'] * (out_resized_levels + 1)  # mse
atrous_unet.model.compile(optimizer=optimizer, loss=losses, metrics=['categorical_accuracy'])

### Train model

In [0]:
class_weight = {key:1/segmentation_classes if key < segmentation_classes
                else 0 for key in range(segmentation_classes + 1)}

history = atrous_unet.model.fit_generator(train_gen,
                                          steps_per_epoch=steps_per_epoch, epochs=epochs,
                                          verbose=1, validation_data=val_gen, validation_steps=val_steps,
                                          class_weight=class_weight)

In [0]:
atrous_unet.model.save('atrous_unet_model_weights_7_epochs.h5')
files.download('atrous_unet_model_weights_7_epochs.h5')

In [0]:
# atrous_unet = AtrousUnet(img_shape, filters, segmentation_classes,
#                          steps, out_resized_levels, kernel_sizes)
# atrous_unet.build_model()
model = load_model('atrous_unet_model_weights_10_epochs.h5')

In [0]:
def get_cmap_dict(reversed=False):
  def color_map(N=256, normalized=False):
    
    def bitget(byteval, idx):
        return ((byteval & (1 << idx)) != 0)

    dtype = 'float32' if normalized else 'uint8'
    cmap = np.zeros((N, 3), dtype=dtype)
    
    for i in range(N):
        r = g = b = 0
        c = i
        for j in range(8):
            r = r | (bitget(c, 0) << 7-j)
            g = g | (bitget(c, 1) << 7-j)
            b = b | (bitget(c, 2) << 7-j)
            c = c >> 3
        cmap[i] = np.array([r, g, b])
    cmap = cmap/255 if normalized else cmap
    return cmap
  
  cmap = color_map()
  cmap_dict = dict()
  for i in range(cmap.shape[0]):
    if reversed:
      cmap_dict[i] = cmap[i]
    else:
      cmap_dict[tuple(cmap[i].astype(np.int64))] = i
  return cmap_dict

In [0]:
def get_images_from_predictions(preds):
  cmap_dict = get_cmap_dict(reversed=True)
  preds = np.argmax(preds, axis=-1)
  imgs = np.zeros((*preds.shape, 3))
  for i, pred in enumerate(preds):
    for j in range(pred.shape[0]):
      for k in range(pred.shape[1]):
        imgs[i, j, k, :] = cmap_dict[pred[j, k]]
  
  return imgs

In [0]:
imgs, labels = val_gen.__next__()
predictions = atrous_unet.model.predict_on_batch(imgs)
pred_labels = predictions[-1]
pred_labels = get_images_from_predictions(pred_labels)

real_labels = labels[-1]
real_labels = get_images_from_predictions(real_labels)

In [0]:
for img, real_label, pred_label in zip(imgs, real_labels, pred_labels):
  image = np.concatenate((img, real_label, pred_label), axis=1).astype(np.uint8)
  print(np.max(image))
#   image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  plt.imshow(image)
  plt.axis('off')
  plt.show()